In [80]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [81]:
# Scrape Huffington (limited history = 15 pages --> 260 articles)
# Set up arrays 
article_title = []
article_URL = []

def scrapeTest(url):
    html_doc = requests.get(url,  headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(html_doc.text, 'html.parser')
    articleDataFrame = []
    wordDataFrame = [] #this is going to be the unorganized word DF

    articleList = soup.find('div', {'class': 'a-page__column--center'})
    articleList = articleList.find("div", {"class": "zone__content"})
    articles = articleList.findAll("div", {"class": "card"})

    for article in articles:
            linkHTML = article.find("a", {"class": "card__link yr-card-headline"})
            link = linkHTML['href']
            linkURL = "https://www.huffingtonpost.com" + link
            articleTitle = article.find('div', {'class': 'card__headline__text'}).text
            begin_url = 'https://www.huffingtonpost.com/entry/'
            cleaned_title = linkURL[linkURL.index(begin_url)+len(begin_url):linkURL.index("_")]
            cleaned_title = re.sub("-", " ", cleaned_title)
            article_title.append(cleaned_title)
            article_URL.append(linkURL)

for x in range (2, 15):
    url = 'https://www.huffingtonpost.com/section/politics?page=' + str(x)
    scrapeTest(url)



In [82]:
# Get time stamp and comment count of each article 
time_stamp = []
comment_count = []


def getTimeComments(url): 
    single_req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(single_req.text, 'html.parser')
    time = soup.find('span', {'class' : "timestamp__date--published"}).text
    time_stamp.append(time)
#   Where you would get the comments here
#   comments = soup.find('span', {'class' : "param-0 param-messagesCount"}).text??
#   comment_count.append(comments)
    
for link in article_URL: 
    getTimeComments(link)

In [83]:
# chrome_path = "./chromedriver"
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome('/Users/doreennguyen/Desktop/news-data-scrape/chromedriver')

In [84]:
# Set up dataframe 
d = {"title" : article_title, "url" : article_URL, "time_stamp" : time_stamp}
articleDF = pd.DataFrame(data=d)
articleDF

,title,url,time_stamp
0,fox news migrant invasion,https://www.huffingtonpost.com/entry/fox-news-...,12/09/2018 07:48 pm ET
1,fox news andrew mccarthy trump indicted,https://www.huffingtonpost.com/entry/fox-news-...,12/09/2018 04:04 pm ET
2,nick ayers decline trump chief staff,https://www.huffingtonpost.com/entry/nick-ayer...,12/09/2018 05:07 pm ET
3,rand paul william barr,https://www.huffingtonpost.com/entry/rand-paul...,12/09/2018 01:07 pm ET
4,trump first wanted his attorney general pick w...,https://www.huffingtonpost.com/entry/trump-fir...,12/09/2018 03:19 pm ET
5,jared kushner saudi crown prince khashoggi,https://www.huffingtonpost.com/entry/jared-kus...,12/09/2018 08:21 am ET
6,marco rubio trump manafort pardon,https://www.huffingtonpost.com/entry/marco-rub...,12/09/2018 11:27 am ET
7,tony evers scott walker legislature,https://www.huffingtonpost.com/entry/tony-ever...,12/09/2018 12:12 pm ET
8,cohen manafort trump nadler impeachment,https://www.huffingtonpost.com/entry/cohen-man...,12/09/2018 12:05 pm ET
9,trump bashed over twitter trolling obama for c...,https://www.huffingtonpost.com/entry/trump-bas...,12/08/2018 10:57 pm ET
